# import

In [1]:
from pathlib import Path
import yt_dlp
import re
import json
from dotenv import load_dotenv
from google import genai
import os

# 関数定義

In [2]:
# LLMによる生成
def LLM_gen(contents: str) -> str:
    client = genai.Client()

    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents= contents
    )
    return response.text
# 危険な文字の除去
def replace_chars(s: str) -> str:
    remove_chars = '\\/:*?"<>|￥＜＞｜'  # 削除対象文字
    table = str.maketrans({ch: '-' for ch in remove_chars})
    s2 = s.translate(table)
    s3 = s2.replace(" ","")
    return s3
# srtファイルを削除したときにjsonファイルのdoneを全てfalseに修正

# youtubeの読み込み

In [3]:
# jsonファイルの読み込み
youtube_links_file_name = "youtube_links.json"
youtube_links_path = Path(youtube_links_file_name)
# if not youtube_links_path.exists():
#     raise 
json_text = youtube_links_path.read_text(encoding="utf-8")
data = json.loads(json_text)

ydl_opts = {
    "skip_download": True,
    "writesubtitles": True,
    "writeautomaticsub": True,
    "subtitleslangs": ["ja"],     # 日本語字幕
    "subtitlesformat": "srt",     # 形式
    "outtmpl": "captions/%(title)s.%(ext)s" 
}

# 文字お越しの読み込み
for v in data:
    print(v)
    url = v['url']
    done = v['done']
    title = v['title']
    
    if done and title!=None:
        print(f"文字お越し済み {title}")
        continue
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ret = ydl.download([url])
        info = ydl.extract_info(url, download=False)
        title = info['title']
        if ret == 0: # 成功した場合
            v['done'] = True
            v['title'] = title
            print(f"[OK] {url} {title}")
        else:
            print(f"[FAIL] {url} {title}")

youtube_links_path.write_text(json.dumps(data, indent=2, ensure_ascii=False), encoding="utf-8")

{'url': 'https://www.youtube.com/watch?v=BBe1ij0AyTk', 'done': True, 'title': '【就活考察】東大生の就職志望No1-コンサルティングファーム-歴史と未来', 'LLM_gen': False}
文字お越し済み 【就活考察】東大生の就職志望No1-コンサルティングファーム-歴史と未来
{'url': 'https://www.youtube.com/watch?v=6GCIwj1X9W4', 'done': True, 'title': 'なぜ鳩のヒナは見かけない？知られざる鳩の生態【アニメ】', 'LLM_gen': False}
文字お越し済み なぜ鳩のヒナは見かけない？知られざる鳩の生態【アニメ】
{'url': 'https://www.youtube.com/watch?v=OP6SEtY7kO8', 'done': True, 'title': '【世界一流エンジニアのチーム作り】成果を上げるチームとは-人に頼るハードルを下げる-同僚と意見が対立したら？-無駄な会議はどんどん削ろう-「理解」ファーストで取り組もう【米マイクロソフト牛尾剛】', 'LLM_gen': False}
文字お越し済み 【世界一流エンジニアのチーム作り】成果を上げるチームとは-人に頼るハードルを下げる-同僚と意見が対立したら？-無駄な会議はどんどん削ろう-「理解」ファーストで取り組もう【米マイクロソフト牛尾剛】
{'url': 'https://www.youtube.com/watch?v=KlZ-QmPteqM', 'done': True, 'title': 'GPTとは何かTransformerの視覚化-Chapter5,DeepLearning', 'LLM_gen': False}
文字お越し済み GPTとは何かTransformerの視覚化-Chapter5,DeepLearning
{'url': 'https://www.youtube.com/watch?v=2Q1rDHtU8Wk', 'done': True, 'title': '【特別編】ハーバードやGoogleで使われている「超効率的学習法」【LIMITLESS超加速学習】', 'LLM_gen': False}
文字お越し済み

870

# 文字お越こし要約(parallel-version)

In [5]:
# ファイル出力なしで要約成功したバージョン
import asyncio 
title_list = []
contents_dic = {} 

# パス設定
caption_dir= Path('captions')
summary_dir = Path('summary')
summary_dir.mkdir(exist_ok=True)
# gemini api keyの読み込み
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# 正規表現を使いtextの抽出とLLMによる要約
text_pattern = r"\d\n.*\n(.*)\n"
for p in caption_dir.iterdir(): 
    p_str = str(p) 
    title_pattern = r"captions\\(.*).ja.srt" 
    title = re.search(title_pattern, p_str)[1] # タイトル部分の抽出 
    replaced_title = replace_chars(title) # 危険文字の変換 
    text = p.read_text(encoding='utf-8') 
    matchs = re.findall(text_pattern, text) 
    text = '\n'.join(matchs) 
    contents= ( 
        "以下はyoutube動画の文字お越しをした文章です。" 
        "要約してください。" 
        "ただし、出力はMarkdown形式のみで行い、" 
        "要約に関係ない説明文や前置きは一切書かないでください。" 
        "見出し・箇条書きを適宜使って整理してください。\n\n" 
        + text 
        ) 
    title_list.append(replaced_title)
    contents_dic[title] = contents
    
async def llm_task(contents: str): 
    loop = asyncio.get_running_loop() 
    return await loop.run_in_executor(None, LLM_gen, contents) 
async def main(): 
    tasks = [llm_task(contents_dic[title]) for title in contents_dic] 
    results = await asyncio.gather(*tasks)

    json_text = youtube_links_path.read_text(encoding="utf-8")
    data = json.loads(json_text)
    if len(data) == len(results):
        print(f"要約した合計は一致")
    else:
        print(f"要約失敗が存在")
        raise RuntimeError

    # youtube_links.jsonに関するループ
    for v in data:
        LLM_gen_flg = False
        name_match_flg = False
        title = v['title']
        # title_listsに関するループ
        for t in title_list:
            if t == v['title']:
                LLM_gen_flg = v['LLM_gen']
                name_match_flg = True
                i = title_list.index(title)
                res = results[i]
                print(f"titleが一致 : {title } : {LLM_gen_flg=},{res}")
                break
            
                v['LLM_gen'] = True
            
        if not name_match_flg:
            print(f"titleが不一致 : {title}")
            break

        if LLM_gen_flg==False:
            print(f"要約を出力します : {title}")
            file_name = summary_dir / f"{title}.md"
            print(file_name.name)
            file_name.write_text(res,encoding='utf-8')
            v['LLM_gen'] = True
        else:
            print(f"要約出力済みなのでスキップ :  {title}")
    youtube_links_path.write_text(json.dumps(data, indent=2, ensure_ascii=False), encoding="utf-8")
await main()

要約した合計は一致
titleが一致 : 【就活考察】東大生の就職志望No1-コンサルティングファーム-歴史と未来 : LLM_gen_flg=False,# コンサルティング業界の現状と人気

*   **高偏差値層からの高い人気**
    *   東大生・京大生の就職人気ランキングにおいて、上位20社中14社をコンサルティングファームが占める。
    *   東大生の約16%がコンサルティング・シンクタンク業界を志望（IT・通信業界の約17%に次ぐ）。
    *   一般的な大学生のコンサル志望率（1.4%）と比較して顕著な差があり、高偏差値層に特有の現象。
    *   「秀才の集団が論理的思考で問題解決をする」というイメージが先行。

# コンサルティングファームの定義と類型

*   **定義**
    *   企業の業務課題を指摘し、対応策や改善策を提案する会社の総称。
    *   企業の依頼を受け、戦略立案や業務改善を行うプロフェッショナル集団。
*   **主要な系統**
    *   **戦略系：** 事業戦略を主に取り扱い、業界の先駆け。
    *   **総合系：** 会計事務所をルーツに持ち、フルラインのサービス（組織、人事、IT戦略など）を提供。
    *   **業務IT系：** 高い技術力を誇り、IT化の波に乗って台頭。
    *   現在は各社がコンサルティング領域を広げているため、系統による分類の意味は薄れているが、ルーツを知ることは興味深い。
*   **日本独自のファーム**
    *   シンクタンク系や国内独立系ファームも存在。外資系ファームとは別の流れで発展。

# コンサルティング業界の歴史

*   **起源（19世紀後半・アメリカ）**
    *   フレデリック・テイラーによる「科学的管理法」（生産現場の効率化、労働コスト削減）がコンサルティング業の起源とされる。
    *   当初は個人の職業だったが、共同事務所（ファーム）が誕生。世界初のファームは1886年設立のアーサー・D・リトル。
    *   初期は事業の効率化を目指すコンサルティングが主流。
*   **発展期（第一次世界大戦後〜世界大恐慌）**
    *   第一次世界大戦後、「永遠の繁栄」と呼ばれたアメリカ経済の中心